<a href="https://colab.research.google.com/github/tkashdat/Open_Project_Spring_2022/blob/master/tiffany_main_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ----------------------------------------------------------------------------------------------------------------------
# Name:         tiffany_main_notebook.ipynb
# Purpose:      Open Project Summer - 2022: Predict mortality with medical data
#               CodaLab competition: https://competitions.codalab.org/competitions/27605#learn_the_details
#
# Author(s):    Tiffany Kashima
#
# Created:      6/20/2022
# Updated:      7/21/2022
# Update Comment(s):
#
# TO DO:
#
# FUTURE WORK:
#
# BUGS TO FIX:
#
# ----------------------------------------------------------------------------------------------------------------------
import numpy as np
import pandas as pd
import time
from sklearn.feature_selection import chi2, f_classif

In [72]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import random as python_random

np.random.seed(123)
tf.random.set_seed(123)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [98]:
# Import training data and feature labels, target

feature_labels = pd.read_csv('/content/mimic_synthetic_feat.csv', header=None).squeeze()
all_train = pd.read_csv('/content/mimic_synthetic_train.csv', delimiter=' ', header=None, names=feature_labels)


In [99]:
all_train

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,GENDER,...,Eyeinfectn,Othereyedx,Otitismedia,Dizziness,Othereardx,Othnervdx,Hrtvalvedx,Carditis,HTN,Htncomplicn
0,45524,108398,2128-05-15_23:42:00,2132-07-23_15:00:00,Private,ENGL,CATHOLIC,DIVORCED,WHITE,F,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,50520,186416,2134-03-17_03:59:00,2113-03-06_12:05:00,Private,ENGL,UNOBTAINABLE,SINGLE,WHITE,M,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,17956,126413,2164-04-05_17:32:00,2180-09-20_16:30:00,Medicaid,SPAN,CATHOLIC,WIDOWED,OTHER,M,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,90258,109355,2102-09-08_00:58:00,2166-06-26_15:30:00,Medicare,ENGL,NOT_SPECIFIED,MARRIED,WHITE,M,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,25159,123784,2163-08-06_12:07:00,2147-01-14_18:40:00,Medicare,ENGL,UNOBTAINABLE,MARRIED,UNKNOWN/NOT_SPECIFIED,F,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29455,75493,158046,2200-01-23_16:16:00,2123-04-10_14:19:00,Medicare,POLI,CATHOLIC,MARRIED,WHITE,F,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
29456,53007,139270,2153-12-07_00:38:00,2179-05-23_13:30:00,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,F,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
29457,73575,159568,2200-06-18_07:15:00,2195-01-15_17:00:00,Private,ENGL,CATHOLIC,MARRIED,WHITE,F,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29458,2722,152443,2179-06-08_07:15:00,2109-10-24_15:18:00,Medicare,ENGL,PROTESTANT_QUAKER,WIDOWED,WHITE,M,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
